In [2]:
from  tensorflow.keras.datasets import mnist
from matplotlib import pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

In [3]:
# A method to download and train dataset

def load_dataset():
    (trainX, trainY),(testX, testY)= mnist.load_data()
    trainX = trainX.reshape((trainX.shape[0],28,28,1))
    testX = testX.reshape((testX.shape[0], 28,28,1))
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

In [4]:
# Normalization
def prep_pixels(train, test):
    train_norm = train.astype("float32")
    test_norm = test.astype("float32")
    train_norm = train_norm/255.0
    test_norm = test_nmorm/255.0
    
    return test_norm ,train_norm

In [8]:
# Define CNN model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation = "relu", kernel_initializer = "he_uniform",
                    input_shape = (28,28,1)))
    model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(100, activation = "relu", kernel_initializer = "he_uniform"))
    model.add(Dense(10, activation = "softmax"))
    opt = SGD(learning_rate = .001, momentum = .9)
    model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ["accuracy"])
    return model

In [9]:
# Evaluation of the model
def evaluate_model(dataX, dataY, n_folds=5):
    scores, histories = [], []
    kfolds = KFold(n_folds, shuffle=True, random_state=1904)
    
    for train_ix, test_ix in kfolds.split(dataX):
        model = define_model()
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        history = model.fit(trainX, trainY, epochs=10, batch_size=32, 
                            validation_data=(testX, testY), verbose=1)
        _, acc = model.evaluate(testX, testY, verbose=1)
        print("> %.3f" %(acc*100.0))
        scores.append(acc)
        histories.append(history)
    return scores, histories
        

In [12]:
# Visualisation - Learning Curves

def summarize_diagnostics(histories):
    for i in range(len(histories)):
        plt.subplot(2, 1, 1)
        plt.title('Cross Entropy Loss')
        plt.plot(histories[i].history['loss'], color='blue', label='train')
        plt.plot(histories[i].history['val_loss'], color='orange', label='test')
        
        plt.subplot(2, 1, 2)
        plt.title("Classification Accuracy")
        plt.plot(histories[i].history['accuracy'], color='blue', label='train')
        plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
    plt.show()     

In [13]:
## summarize Model Performance
def summarize_performance(scores):
    print("Accuracy: mean=%.3f std=%.3f, n=%d" %(mean(scores)*100, std(scores)*100, len(scores)))
    plt.boxplot(scores)
    plt.show()

In [14]:
## Final method to train, evaluate and summarize the performance
def run_test_harness():
    trainX, trainY, testX, testY = load_dataset()
    trainX, testX = prep_pixels(trainX, testX)
    scores, histories = evaluate_model(trainX, trainY)
    summarize_diagnostics(histories)
    summarize_diagnostics(scores)